<a href="https://colab.research.google.com/github/Narazaka/voice-changer/blob/colab-rvc/VoiceChangerDemo_Simple.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

MMVCプレイヤー（超簡単版）
---

このノートはColab上でMMVCのボイチェンを行うノートです。

正式版はローカルPC上で動かすアプリケーションです。

正式版は、多くの場合より少ないタイムラグで滑らかに音声を変換できます。

詳細な使用方法はこちらの[リポジトリ](https://github.com/w-okada/voice-changer)からご確認ください。


# GPUを確認
GPUを用いたほうが高速に処理が行えます。

下記のコマンドでGPUが確認できない場合は、上のメニューから

「ランタイム」→「ランタイムの変更」→「ハードウェア アクセラレータ」

でGPUを選択してください。

In [1]:
# (1) GPUの確認
!nvidia-smi

Wed Apr 12 11:15:02 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   60C    P8    11W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# リポジトリのクローン
リポジトリをクローンします

In [2]:
# (2) リポジトリのクローン
!git clone --depth 1 https://github.com/w-okada/voice-changer.git  -b master
%cd voice-changer/server
!curl -LO https://huggingface.co/lj1995/VoiceConversionWebUI/resolve/main/hubert_base.pt
!git clone https://github.com/isletennos/MMVC_Client.git
!cd MMVC_Client && git checkout 04f3fec4fd82dea6657026ec4e1cd80fb29a415c && cd -


Cloning into 'voice-changer'...
remote: Enumerating objects: 354, done.
remote: Counting objects: 100% (354/354), done.
remote: Compressing objects: 100% (293/293), done.
remote: Total 354 (delta 91), reused 193 (delta 44), pack-reused 0
Receiving objects: 100% (354/354), 15.16 MiB | 13.36 MiB/s, done.
Resolving deltas: 100% (91/91), done.
/content/voice-changer/server
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  1131  100  1131    0     0   8767      0 --:--:-- --:--:-- --:--:--  8767
100  180M  100  180M    0     0   177M      0  0:00:01  0:00:01 --:--:--  213M
Cloning into 'MMVC_Client'...
remote: Enumerating objects: 719, done.
remote: Counting objects: 100% (447/447), done.
remote: Compressing objects: 100% (146/146), done.
remote: Total 719 (delta 315), reused 369 (delta 297), pack-reused 272
Receiving objects: 100% (719/719), 808.88 KiB | 19.73 MiB/s, done.
Resol

# モジュールのインストール

必要なモジュールをインストールします。

In [25]:
import os

if not os.path.exists("/usr/local/bin/conda"):
    ! curl -O https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh
    ! chmod +x Miniconda3-latest-Linux-x86_64.sh
    ! bash ./Miniconda3-latest-Linux-x86_64.sh -b -f -p /usr/local
    ! rm Miniconda3-latest-Linux-x86_64.sh


In [26]:
%%bash
eval "$(conda shell.bash hook)"
conda create -y -n mmvc-server python=3.10
conda activate mmvc-server

Solving environment: ...working... done

## Package Plan ##

  environment location: /usr/local/envs/mmvc-server

  added / updated specs:
    - python=3.10


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    openssl-1.1.1t             |       h7f8727e_0         3.7 MB
    pip-23.0.1                 |  py310h06a4308_0         2.6 MB
    python-3.10.10             |       h7a1cb2a_2        26.9 MB
    sqlite-3.41.1              |       h5eee18b_0         1.2 MB
    tzdata-2023c               |       h04d1e81_0         116 KB
    wheel-0.38.4               |  py310h06a4308_0          64 KB
    ------------------------------------------------------------
                                           Total:        34.6 MB

The following NEW packages will be INSTALLED:

  _libgcc_mutex      pkgs/main/linux-64::_libgcc_mutex-0.1-main 
  _openmp_mutex      pkgs/main/linux-64::_openmp_mutex-5.1-1_gnu



==> WARNING: A newer version of conda exists. <==
  current version: 23.1.0
  latest version: 23.3.1

Please update conda by running

    $ conda update -n base -c defaults conda

Or to minimize the number of packages updated during conda update use

     conda install conda=23.3.1




In [ ]:
# (3) 設定ファイルの確認
!apt-get install -y libsndfile1-dev
!pip install -r requirements.txt


Reading package lists... Done
Building dependency tree       
Reading state information... Done
libsndfile1-dev is already the newest version (1.0.28-7ubuntu0.1).
0 upgraded, 0 newly installed, 0 to remove and 24 not upgraded.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.8/57.8 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.3/57.3 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 62.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 887.5/887.5 MB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 102.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 101.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 253.0/253.0 kB 33.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 214.4/214.4 kB 29.4 MB/s eta 0:00:00
  Installing build dependenci

# サーバの起動

サーバを起動します。(4-1)

サーバの起動状況を確認します。(4-2) 

このセルは繰り返し実行することになるのでCtrl+Retでセルを実行してください。

下記のようなテキストが表示されたら起動完了です。

**`VoiceChanger Initialized`**

```
    Voice Changerを起動しています。
    -- 設定 -- 
    CONFIG:None, MODEL:None ONNX_MODEL:None
```



In [ ]:
# (4-1) サーバの起動
import random
PORT = 10000 + random.randint(1, 9999)
LOG_FILE = f"LOG_FILE_{PORT}"

get_ipython().system_raw(f'python3 MMVCServerSIO.py -t MMVC -p {PORT} --https False --colab True >{LOG_FILE} 2>&1 &')
#print(f"PORT:{PORT}, LOG_FILE:{LOG_FILE}")

In [ ]:
# (4-2) サーバの起動確認
!sleep 30
!tail -20 {LOG_FILE}

VoiceChanger Initialized (GPU_NUM:1, mps_enabled:False)
    Voice Changerを起動しています。
    -- 設定 -- 
    CONFIG:None, MODEL:None ONNX_MODEL:None


# プロキシを起動
ウェブサーバへのアクセスをするためのプロキシを起動します。

表示されたURLをクリックして開くと別タブでアプリが開きます。

Colabなので、ロードにある程度時間がかかります(30秒くらい)。

In [ ]:
# (5) プロキシを起動
from google.colab.output import eval_js
proxy = eval_js( "google.colab.kernel.proxyPort(" + str(PORT) + ")" )
print(f"{proxy}front/?colab=true")

https://xzok7khbzs-496ff2e9c6d22116-13279-colab.googleusercontent.com/front/?colab=true
